In [110]:
from opt.ml.Dataset import ImageDataset
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset


from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [122]:
check = torch.load('./opt/ml/checkpoints/F1CE_additional/checkpoint_at_7_additional_F1CE.pickle')

In [123]:
from opt.ml.models import Resnet34, Resnext50, Resnet101, Resnet101_8class

model = Resnet101_8class(num_classes=8)

In [124]:
model.load_state_dict(check['model_state_dict'])

<All keys matched successfully>

In [100]:
model.load_state_dict(torch.load('./opt/ml/learned_models/agefilter_F1CE_state_dict.pt'))

<All keys matched successfully>

In [89]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [125]:
#submissiona 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv('./input/data/eval/info.csv')
test_path = './input/data/eval/images/'
test_data_path = [os.path.join(test_path, img) for img in submission.ImageID]


test_transform = transforms.Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2))

])

test_dataset = TestDataset(test_data_path, 
                           transform= test_transform)

test_data_loader = DataLoader(test_dataset,
                         shuffle=False)




# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model.to(device)
model.eval()

indexing_dict = {'gender': [0, 3],
                 'age': [0, 1, 2],
                 'mask':[0, 6, 12]}

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for batch_images in tqdm(test_data_loader):
    with torch.no_grad():
        images = batch_images.to(device)
        pred = model(images)
        age = pred[0][:2].argmax()
        gender = pred[0][2:5].argmax()
        mask = pred[0][5:].argmax()
        
        classified = (indexing_dict['gender'][int(age.cpu())] + 
                     indexing_dict['age'][int(gender.cpu())] + 
                     indexing_dict['mask'][int(mask.cpu())])


#         pred = pred.argmax(dim=-1)
#         classified = pred.cpu().numpy()[0]
        all_predictions.append(classified)
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv('prediction_csv/CEadditional_7_epochs.csv', index=False)
print('test inference is done!')


test inference is done!


In [145]:
from glob import glob

df_list = []
for i, df in enumerate(glob('./candidate/*.csv')):
    df_list.append(pd.read_csv(df))
    df_list[i].columns = ['ImageID', i]

In [150]:
all_pred = df_list[0]

for df in df_list[1:]:
    all_pred = pd.merge(all_pred, df, how= 'left')

In [186]:
from scipy.stats import mode
ensemble = []
for values in all_pred.values:
    ensemble.append(mode(values[1:])[0][0])

In [189]:
submission['ans'] = pd.Series(ensemble)

In [191]:
submission.to_csv('prediction_csv/ensemble.csv', index= False)